# <font color='red'>**ATENÇÃO**

Para a execução desse notebook será necessária a "montagem" do google drive para acesso de leitura e gravação de arquivos.
A variável **`path`** deve ser preenchida com o valor do diretório que estiver localizado o notebook.
</font>

# Análise histórico de medalhas olímpicas
Este notebook analisará o histórico de medalhas olímpicas distribuídas entre os atletas.
O dataset utilizado está disponível em https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results, segundo o autor, obtido via scrap em https://www.sports-reference.com/.

In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/Processamento de Dados Massivos/Laboratorios/MapReduce/'

Primeiramente uma rápida inspeção nos dados:

In [ ]:
atletas = pd.read_csv(path + 'athlete_events.csv')

In [ ]:
atletas[atletas.Team == 'Brazil']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
665,386,Alexandre Abeid,M,22.0,194.0,92.0,Brazil,BRA,1972 Summer,1972,Summer,Munich,Volleyball,Volleyball Men's Volleyball,NaN
666,386,Alexandre Abeid,M,26.0,194.0,92.0,Brazil,BRA,1976 Summer,1976,Summer,Montreal,Volleyball,Volleyball Men's Volleyball,NaN
668,388,Abel Carlos da Silva Braga,M,19.0,190.0,73.0,Brazil,BRA,1972 Summer,1972,Summer,Munich,Football,Football Men's Football,NaN
781,451,Diana Monteiro Abla,F,21.0,175.0,75.0,Brazil,BRA,2016 Summer,2016,Summer,Rio de Janeiro,Water Polo,Water Polo Women's Water Polo,NaN
1005,565,Glauclio Serro Abreu,M,26.0,185.0,75.0,Brazil,BRA,2004 Summer,2004,Summer,Athina,Boxing,Boxing Men's Middleweight,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268652,134400,"Maria Jos ""Zz"" Bertolotti",F,26.0,190.0,NaN,Brazil,BRA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Women's Basketball,NaN
269745,134976,Sieglinda Lenk Zigler,F,17.0,NaN,NaN,Brazil,BRA,1936 Summer,1936,Summer,Berlin,Swimming,Swimming Women's 100 metres Backstroke,NaN
269899,135037,Moacir Zimmermann,M,32.0,184.0,70.0,Brazil,BRA,2016 Summer,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's 20 kilometres Walk,NaN
270478,135289,Zzimo Alves Calazans,M,20.0,NaN,NaN,Brazil,BRA,1952 Summer,1952,Summer,Helsinki,Football,Football Men's Football,NaN


In [ ]:
atletas.count()

ID        271116
Name      271116
Sex       271116
Age       261642
Height    210945
Weight    208241
Team      271116
NOC       271116
Games     271116
Year      271116
Season    271116
City      271116
Sport     271116
Event     271116
Medal      39783
dtype: int64

O primeiro ponto será obter a ordenação entre os países de acordo com o número de atletas que já participaram dos jogos.
Para isso, a função `map` vai tratar o dataframe original e criar os pares "chave e valor" para cada linha.

In [ ]:
# criação da tupla (país, 1)
t = [(i, 1) for i in atletas['Team']]

In [ ]:
for item in t:
  print('%s\t%s'% (item[0], item[1]))

In [ ]:
def map_paises_atletas(df):
  pais_atleta = [(i, 1) for i in df['Team']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in pais_atleta:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()
  

In [ ]:
map_paises_atletas(atletas)

Usando o mesmo método **shuffle** implementado nos demais notebooks. 

In [ ]:
from collections import defaultdict

def shuffle(path, nome_arquivo_mapper):
  resultado_intermediario = defaultdict(list)
  with open(path + nome_arquivo_mapper) as arquivo_entrada:
    for linha in arquivo_entrada:
      chave, valor = linha.split('\t')
      resultado_intermediario[chave].append(int(valor))
  arquivo_entrada.close()
  resultado = dict(resultado_intermediario)
  arquivo_shuffle = open(path + "arquivo_saida_shuffle_medalhas.txt", 'w')
  arquivo_shuffle.write(str(resultado))
  arquivo_shuffle.close()

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )

Usando o mesmo método **reducer** já implementado anteriormente, usando a função para somar os elementos de um array. 

In [ ]:
from functools import reduce

def somar_elementos(x,y):
  return x+y
  
def reducer(path, nome_arquivo_shuffle, funcao_reduce):
  arquivo = open(path + nome_arquivo_shuffle,'r')
  conteudo_entrada = arquivo.read()
  dict_entrada = eval(conteudo_entrada)
  arquivo.close()
  dict_result = {}
  for chave, lista in dict_entrada.items():
    dict_result[chave] = reduce(funcao_reduce, lista)
  arquivo_reduce = open(path + "arquivo_saida_reduce_medalhas.txt", 'w')
  for item  in sorted(dict_result, key = dict_result.get, reverse = True):
    arquivo_reduce.write(item + ' ' + str(dict_result[item]))
    arquivo_reduce.write("\n")
  arquivo_reduce.close()

In [ ]:
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos )

Agora podem ser exploradas outras questões, como:
1. Qual foi o esporte com maior número de atletas até o momento??
1. Qual foi a edição dos jogos que mais contou com atletas participantes??
1. Qual país possui o maior número de atletas medalhistas??
1. Qual o esporte que distribuiu o maior número de medalhas??
1. Em qual o esporte o Brasil possui o maior número de medalhas??
1. Quais outras perguntas poderiam ser feitas?? Liste mais 3 juntamente com o código das funções `mapper` e `reducer` necessárias.

## 1. Qual foi o esporte com maior número de atletas até o momento??

Para esse tópico, a função mapper deverá ser alterada para montar o par **<esporte,1>**

In [ ]:
def map_esporte_atleta(df):
  esporte_atleta = [(i, 1) for i in df['Sport']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in esporte_atleta:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()

In [ ]:
map_esporte_atleta(atletas)

A função `shuffle` como sempre, não precisa ser ajustada, pois representaria o papel da infra-estrutura da biblioteca que implementa o modelo MapReduce. 

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )

A função `reducer`, também seguirá o padrão de somar os valores, e não precisará ser alterada com relação oa tópico anterior.

In [ ]:
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos )

Conferindo a nova versão do arquivo **`arquivo_saida_reduce_medalhas`** verifica-se que o esporte que mais possui atletas olpimpicos é o Atletismo com 38624, seguido pela Ginástica com 26707 atletas.

## 2. Qual foi a edição dos jogos que mais contou com atletas participantes??

Para esse tópico, a função mapper deverá ser alterada para montar o par **<jogos, 1>**

In [ ]:
def map_edicao_atleta(df):
  esporte_atleta = [(i, 1) for i in df['Games']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in esporte_atleta:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()

In [ ]:
map_edicao_atleta(atletas)

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos )

Conferindo a nova versão do arquivo **`arquivo_saida_reduce_medalhas`** verifica-se que a edição de jogos que mais possuiu atletas participantes foi a edição dos Jogos Olímpicos de Verão de 2000, com 13821 atletas, disputados em Atenas, seguido da edição disputada em 1996 em Atlanta, que contou com 13780 atletas.

## 3. Qual país possui o maior número de atletas medalhistas??

Nesse tópico será aplicado um filtro prévio 
sobre o dataframe `atletas` para restar apenas os atletas medalhistas. A função mapper ainda deverá ser alterada para montar o par **<Time,1>**

In [ ]:
def map_pais_atleta_medalhista(df):
  df = df[df.Medal.notna()]
  pais_atleta_medalhista = [(i, 1) for i in df['Team']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in pais_atleta_medalhista:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()

In [ ]:
map_pais_atleta_medalhista(atletas)

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos )

Conferindo a nova versão do arquivo **`arquivo_saida_reduce_medalhas`** verifica-se que o país que mais conquistou medalhas foi os Estados Unidos com 5219.

## 4. Qual o esporte que distribuiu o maior número de medalhas??

Nesse tópico, a função mapper deverá ser alterada para montar o par **<Sport,1>**, realizando o filtro prévio dos registros de atletas medalhistas.

In [ ]:
def map_esporte_atleta_medalhista(df):
  df = df[df.Medal.notna()]
  esporte_atleta_medalhista = [(i, 1) for i in df['Sport']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in esporte_atleta_medalhista:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()

In [ ]:
map_esporte_atleta_medalhista(atletas)

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos)

Conferindo a nova versão do arquivo **`arquivo_saida_reduce_medalhas`** verifica-se que o esporte que mais distribuiu medalhas olímpicas é o Atletismo com 3969, seguido pela Natação com 3048.

## 5. Em qual o esporte o Brasil possui o maior número de medalhas??

Para esse item, o dataset será previamente filtrado para considerar somente os atletas medalhistas brasileiros.

In [ ]:
def map_esporte_atleta_medalhista_brasileiro(df):
  df = df[(df.Medal.notna()) & (df.Team == 'Brazil')]
  esporte_atleta_medalhista = [(i, 1) for i in df['Sport']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in esporte_atleta_medalhista:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()

In [ ]:
map_esporte_atleta_medalhista_brasileiro(atletas) 

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos)

Conferindo a nova versão do arquivo **`arquivo_saida_reduce_medalhas`** verifica-se que o esporte com mais medalhistas brasileiros é o Futebol com 136 atletas, devido ao tamanho dos elencos. O segundo é o Voleibol, com 120.

## 6. Qual país ganhou o maior número de medalhas de ouro?? 

## 7. Com qual idade os atletas ganharam o maior número de medalhas??

In [ ]:
def map_idade_atleta_medalhista(df):
  df = df[(df.Medal.notna())]
  esporte_atleta_medalhista = [(i, 1) for i in df['Age']]
  arquivo = open(path + "arquivo_saida_mapper_medalhas.txt", 'w')
  for item in esporte_atleta_medalhista:
    arquivo.write('%s\t%s\n'% (item[0], item[1]))
  arquivo.close()

In [ ]:
map_idade_atleta_medalhista(atletas)

In [ ]:
shuffle(path,'arquivo_saida_mapper_medalhas.txt' )
reducer(path,'arquivo_saida_shuffle_medalhas.txt', somar_elementos)